In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from fbprophet import Prophet

ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.


In [3]:
features = pd.read_csv('walmart-recruiting-store-sales-forecasting/features.csv',parse_dates=[1])
stores = pd.read_csv('walmart-recruiting-store-sales-forecasting/stores.csv')
test = pd.read_csv('walmart-recruiting-store-sales-forecasting/test.csv',parse_dates=[2])
train = pd.read_csv('walmart-recruiting-store-sales-forecasting/train.csv',parse_dates=[2])

In [4]:
train_data = pd.merge(left=pd.merge(left=train,right=stores,how='left'),right=features,how='left',on=['Store','Date','IsHoliday'])
test_data = pd.merge(left = pd.merge(left=test,right=stores,how='left'),right=features,how='left',on = ['Store','Date','IsHoliday'])

In [5]:
sb_dates = pd.to_datetime(['2010-02-12','2011-02-11','2012-02-10','2013-02-08'])
labor_dates = pd.to_datetime(['2010-09-10','2011-09-09','2012-09-07','2013-09-06'])
thanksgiving_dates = pd.to_datetime(['2010-11-26','2011-11-25','2012-11-23','2013-11-29'])
christmas_dates = pd.to_datetime(['2010-12-31','2011-12-30','2012-12-28','2013-12-27'])


super_bowl = pd.DataFrame({'holiday':'SuperBowl',
                          'ds':sb_dates})
labor_day = pd.DataFrame({'holiday':'LaborDay',
                         'ds':labor_dates})
thanksgiving = pd.DataFrame({'holiday':'Thanksgiving',
                            'ds':thanksgiving_dates})
christmas = pd.DataFrame({'holiday':'Christmas',
                         'ds':christmas_dates})

holiday_dates = pd.concat((super_bowl,labor_day,thanksgiving,christmas))

In [6]:
redundant_pairs_mask = (~(train_data.Store.map(str) +'-'+ train_data.Dept.map(str)).isin(test_data.Store.map(str) +'-'+ test_data.Dept.map(str)))

In [7]:
train_data = train_data[~redundant_pairs_mask]

In [8]:
store_dept_pairs = train_data.groupby(['Store','Dept']).size().reset_index()
store_dept_pairs.columns = ['Store','Dept','Count']

less_than_5_train = store_dept_pairs[store_dept_pairs.Count<5]

In [9]:
store_dept_pairs = test_data.groupby(['Store','Dept']).size().reset_index()
store_dept_pairs.columns = ['Store','Dept','Count']

less_than_5_test = store_dept_pairs[store_dept_pairs.Count<5]

In [11]:
store_numbers =  train_data.Store.unique()
all_train_preds = pd.DataFrame({'Date':[],
                                'Store':[],
                                'Dept':[],
                                'preds':[]})
all_test_preds = pd.DataFrame({'Date':[],
                               'Store':[],
                               'Dept':[],
                               'preds':[]})

detailed_data = dict()

for store_number in store_numbers:
    
    dept_numbers = train_data[(train_data.Store==store_number)].Dept.unique()
    
    for dept_number in dept_numbers:
        
        current_data = dict()
        
        dept_data = train_data.loc[((train_data.Store==store_number) & (train_data.Dept==dept_number))]
        dept_sales = train_data[((train_data.Store==store_number) & (train_data.Dept==dept_number))].Weekly_Sales
        test = test_data[((test_data.Store==store_number) & (test_data.Dept==dept_number))]
        
        current_data['dept_data'] = dept_data
        current_data['dept_sales'] = dept_sales
        current_data['test']=test
        
        train_df = pd.DataFrame({'ds':dept_data.Date,'y':dept_sales})

        if(len(train_df)<2):
            print('skipping because len of dept < 2')
            continue
        
        m = Prophet(weekly_seasonality=True,daily_seasonality=False,yearly_seasonality=True,holidays=holiday_dates)
        
        m.fit(train_df)
                
        train_pred = m.predict()
        
        test_df = pd.DataFrame({'ds':test.Date})
        test_pred = m.predict(test_df)
        
        current_data['train_pred'] = train_pred
        current_data['test_pred'] = test_pred
        current_data['model']=m
        
        combined_preds = pd.concat((train_pred,test_pred),ignore_index=True)
        
        current_data['combined_preds'] = combined_preds

        train_preds_for_model = pd.DataFrame({'Date':train_df.ds,
                                              'Store':store_number,
                                              'Dept':dept_number,
                                              'preds':train_pred.yhat})
        
        test_preds_for_model = pd.DataFrame({'Date':test_df.ds,
                                             'Store':store_number,
                                             'Dept':dept_number,
                                             'preds':test_pred.yhat})
        
        all_train_preds = pd.concat((all_train_preds,train_preds_for_model),ignore_index=True)
        all_test_preds = pd.concat((all_test_preds,test_preds_for_model),ignore_index=True)
        
        
        
        detailed_data[str(store_number)+'-'+str(dept_number)] = current_data
                
        print('Store: ',store_number,', Dept: ',dept_number)

Store:  1 , Dept:  1
Store:  1 , Dept:  2
Store:  1 , Dept:  3
Store:  1 , Dept:  4
Store:  1 , Dept:  5
Store:  1 , Dept:  6
Store:  1 , Dept:  7
Store:  1 , Dept:  8
Store:  1 , Dept:  9
Store:  1 , Dept:  10
Store:  1 , Dept:  11
Store:  1 , Dept:  12
Store:  1 , Dept:  13
Store:  1 , Dept:  14
Store:  1 , Dept:  16
Store:  1 , Dept:  17
Store:  1 , Dept:  18
Store:  1 , Dept:  19
Store:  1 , Dept:  20
Store:  1 , Dept:  21
Store:  1 , Dept:  22
Store:  1 , Dept:  23
Store:  1 , Dept:  24
Store:  1 , Dept:  25
Store:  1 , Dept:  26
Store:  1 , Dept:  27
Store:  1 , Dept:  28
Store:  1 , Dept:  29
Store:  1 , Dept:  30
Store:  1 , Dept:  31
Store:  1 , Dept:  32
Store:  1 , Dept:  33
Store:  1 , Dept:  34
Store:  1 , Dept:  35
Store:  1 , Dept:  36
Store:  1 , Dept:  37
Store:  1 , Dept:  38
Store:  1 , Dept:  40
Store:  1 , Dept:  41
Store:  1 , Dept:  42
Store:  1 , Dept:  44
Store:  1 , Dept:  45


INFO:fbprophet:n_changepoints greater than number of observations.Using 16.


Store:  1 , Dept:  46
Store:  1 , Dept:  47
Store:  1 , Dept:  48
Store:  1 , Dept:  49
Store:  1 , Dept:  52
Store:  1 , Dept:  54
Store:  1 , Dept:  55
Store:  1 , Dept:  56
Store:  1 , Dept:  58
Store:  1 , Dept:  59
Store:  1 , Dept:  60
Store:  1 , Dept:  67
Store:  1 , Dept:  71
Store:  1 , Dept:  72
Store:  1 , Dept:  74
Store:  1 , Dept:  79
Store:  1 , Dept:  80
Store:  1 , Dept:  81
Store:  1 , Dept:  82
Store:  1 , Dept:  83
Store:  1 , Dept:  85
Store:  1 , Dept:  87
Store:  1 , Dept:  90
Store:  1 , Dept:  91
Store:  1 , Dept:  92
Store:  1 , Dept:  93
Store:  1 , Dept:  94
Store:  1 , Dept:  95
Store:  1 , Dept:  96
Store:  1 , Dept:  97
Store:  1 , Dept:  98
Store:  1 , Dept:  99
Store:  2 , Dept:  1
Store:  2 , Dept:  2
Store:  2 , Dept:  3
Store:  2 , Dept:  4
Store:  2 , Dept:  5
Store:  2 , Dept:  6
Store:  2 , Dept:  7
Store:  2 , Dept:  8
Store:  2 , Dept:  9
Store:  2 , Dept:  10
Store:  2 , Dept:  11
Store:  2 , Dept:  12
Store:  2 , Dept:  13
Store:  2 , Dept:  

INFO:fbprophet:n_changepoints greater than number of observations.Using 16.


Store:  2 , Dept:  46
Store:  2 , Dept:  47
Store:  2 , Dept:  48
Store:  2 , Dept:  49
Store:  2 , Dept:  52
Store:  2 , Dept:  54
Store:  2 , Dept:  55
Store:  2 , Dept:  56
Store:  2 , Dept:  58
Store:  2 , Dept:  59
Store:  2 , Dept:  60
Store:  2 , Dept:  67
Store:  2 , Dept:  71
Store:  2 , Dept:  72


...

Store:  44 , Dept:  52
Store:  44 , Dept:  55
Store:  44 , Dept:  56
Store:  44 , Dept:  59
Store:  44 , Dept:  60
Store:  44 , Dept:  67
Store:  44 , Dept:  72
Store:  44 , Dept:  74
Store:  44 , Dept:  79
Store:  44 , Dept:  80
Store:  44 , Dept:  81
Store:  44 , Dept:  82
Store:  44 , Dept:  83
Store:  44 , Dept:  85
Store:  44 , Dept:  87
Store:  44 , Dept:  90
Store:  44 , Dept:  91
Store:  44 , Dept:  92
Store:  44 , Dept:  93
Store:  44 , Dept:  94
Store:  44 , Dept:  95
Store:  44 , Dept:  96
Store:  44 , Dept:  97
Store:  44 , Dept:  98
Store:  45 , Dept:  1
Store:  45 , Dept:  2
Store:  45 , Dept:  3
Store:  45 , Dept:  4
Store:  45 , Dept:  5
Store:  45 , Dept:  6
Store:  45 , Dept:  7
Store:  45 , Dept:  8
Store:  45 , Dept:  9
Store:  45 , Dept:  10
Store:  45 , Dept:  11
Store:  45 , Dept:  12
Store:  45 , Dept:  13
Store:  45 , Dept:  14
Store:  45 , Dept:  16
Store:  45 , Dept:  17
Store:  45 , Dept:  18
Store:  45 , Dept:  19
Store:  45 , Dept:  20
Store:  45 , Dept:  

INFO:fbprophet:n_changepoints greater than number of observations.Using 19.


Store:  45 , Dept:  44
Store:  45 , Dept:  45


INFO:fbprophet:n_changepoints greater than number of observations.Using 8.


Store:  45 , Dept:  46
Store:  45 , Dept:  47
Store:  45 , Dept:  49
Store:  45 , Dept:  52
Store:  45 , Dept:  54
Store:  45 , Dept:  55
Store:  45 , Dept:  56
Store:  45 , Dept:  58
Store:  45 , Dept:  59
Store:  45 , Dept:  60
Store:  45 , Dept:  67
Store:  45 , Dept:  71
Store:  45 , Dept:  72
Store:  45 , Dept:  74
Store:  45 , Dept:  79
Store:  45 , Dept:  80
Store:  45 , Dept:  81
Store:  45 , Dept:  82
Store:  45 , Dept:  83
Store:  45 , Dept:  85
Store:  45 , Dept:  87
Store:  45 , Dept:  90
Store:  45 , Dept:  91
Store:  45 , Dept:  92
Store:  45 , Dept:  93
Store:  45 , Dept:  94
Store:  45 , Dept:  95
Store:  45 , Dept:  97
Store:  45 , Dept:  98


In [12]:
all_train_preds = pd.DataFrame({'Date':[],
                                'Store':[],
                                'Dept':[],
                                'preds':[]})

In [14]:
all_train_preds = pd.DataFrame({'Date':[],
                                'Store':[],
                                'Dept':[],
                                'preds':[]})
for i in detailed_data.keys():
    store_no, dept_no = str.split(i,'-')
    
    df = pd.DataFrame({ 'Date':detailed_data[i]['train_pred'].ds,
                        'Store':int(store_no),
                        'Dept':int(dept_no),
                        'preds':detailed_data[i]['train_pred'].yhat})
    
    all_train_preds = pd.concat((all_train_preds,df),ignore_index=True)

In [15]:
len(all_train_preds)

419194

In [16]:
all_train_preds.describe()

,Store,Dept,preds
count,419194.000000,419194.000000,419194.000000
mean,22.193047,44.210058,16071.264901
std,12.791454,30.554016,22567.088709
min,1.000000,1.000000,-10555.768582
25%,11.000000,18.000000,2182.479357
50%,22.000000,37.000000,7742.307429
75%,33.000000,74.000000,20372.099880
max,45.000000,99.000000,666520.667253


In [17]:
all_test_preds = pd.DataFrame({'Date':[],
                                'Store':[],
                                'Dept':[],
                                'preds':[]})
for i in detailed_data.keys():
    store_no, dept_no = str.split(i,'-')
    
    df = pd.DataFrame({ 'Date':detailed_data[i]['test_pred'].ds,
                        'Store':int(store_no),
                        'Dept':int(dept_no),
                        'preds':detailed_data[i]['test_pred'].yhat})
    
    all_test_preds = pd.concat((all_test_preds,df),ignore_index=True)

In [20]:
len(test_data)-len(all_test_preds)

78

In [23]:
?pd.merge

In [240]:
merge_test = pd.merge(test_data,all_test_preds,on=['Store','Dept'],how='outer',indicator=True)

In [241]:
missing_in_test = merge_test[merge_test._merge != 'both']

In [242]:
common = missing_in_test.Store.astype(str)+'-'+missing_in_test.Dept.astype(str)

In [243]:
miss_ind_tr = np.isin(train_data.Store.astype(str)+'-'+train_data.Dept.astype(str),common)
miss_ind_te = np.isin(test_data.Store.astype(str)+'-'+test_data.Dept.astype(str),common)

In [244]:
test_data[miss_ind_te]

,Store,Dept,Date,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
9495,4,39,2013-07-12,False,A,205863,82.30,3.368,6501.43,1464.81,124.97,3809.96,3868.60,NaN,NaN
13300,5,99,2012-11-23,True,B,34875,59.07,3.211,310.95,NaN,43319.43,46.61,372.22,224.156022,5.422
13301,5,99,2012-12-21,False,B,34875,58.63,3.098,1344.90,NaN,15.00,NaN,911.84,224.435138,5.422
13302,5,99,2012-12-28,True,B,34875,47.47,3.108,6052.63,13017.25,60.81,72.80,3503.26,224.555872,5.422
23890,9,99,2012-12-07,False,B,125833,61.19,3.198,2521.31,NaN,388.00,171.98,4047.33,227.456446,4.954
23891,9,99,2013-07-05,False,B,125833,80.11,3.422,5382.87,886.86,484.64,1899.92,1478.67,NaN,NaN
23892,9,99,2013-07-19,False,B,125833,75.26,3.556,1040.56,188.09,NaN,178.85,1507.30,NaN,NaN
23893,9,99,2013-07-26,False,B,125833,82.37,3.620,52.43,258.14,NaN,2.50,1265.60,NaN,NaN
26670,10,99,2012-11-30,False,B,126512,64.13,3.719,4304.00,NaN,8935.00,175.68,12676.61,131.427533,6.943
26671,10,99,2012-12-07,False,B,126512,61.21,3.668,13613.52,NaN,822.00,3516.91,18831.34,131.478400,6.943


In [245]:
all_missing_values_in_test = pd.merge(test_data[miss_ind_te],train_data[miss_ind_tr],on=['Store','Dept'],how='outer',indicator=True)[['Date_x','Store','Dept','Weekly_Sales','_merge']]

In [246]:
missing_stores_mean_sales = train_data[np.isin(train_data.Store,all_missing_values_in_test[all_missing_values_in_test._merge=='left_only'].Store)].groupby('Store')['Weekly_Sales'].mean()

In [249]:
missing_stores_mean_sales

Store
5      5057.804399
9      8790.242338
10    26388.382446
18    15831.094588
24    19120.266266
25    10361.898767
34    13696.676833
36     8613.359460
37    10301.641444
42    11461.570732
45    11711.328228
Name: Weekly_Sales, dtype: float64

In [250]:
a = all_missing_values_in_test.set_index('Store')
a['Weekly_Sales'] = a['Weekly_Sales'].fillna(missing_stores_mean_sales)
a.reset_index(inplace=True)
a.columns = ['Store','Date','Dept','preds','_merge']
a.drop('_merge',inplace=True,axis=1)
a = a[['Date','Store','Dept','preds']]
a = pd.concat((all_test_preds,a),axis=0)

In [252]:
len(a)

115064

In [256]:
test_output = a.copy()
test_output['id'] = test_output['Store'].astype(int).astype(str) + '_' +  test_output['Dept'].astype(int).astype(str) + '_' +  test_output['Date'].astype(str)
test_output = test_output[['id', 'preds']]
test_output = test_output.rename(columns={'id': 'Id', 'preds': 'Weekly_Sales'})

test_output.to_csv('output.csv', index=False)

In [254]:
len(test_output)

115064

In [235]:
len(test_data)

115064

In [255]:
test_output.head()

,Id,Weekly_Sales
0,1.0_1.0_2012-11-02,33204.907992
1,1.0_1.0_2012-11-09,27729.719811
2,1.0_1.0_2012-11-16,19150.108693
3,1.0_1.0_2012-11-23,18883.921246
4,1.0_1.0_2012-11-30,20394.641842
